In [10]:
import import_ipynb
import Tumor_identifier as ti
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import torch
print(torch.cuda.is_available())


False


# Creating a Test(Dummy model for evaluation)
## Using model.eval() to check

In [9]:
mri_dataset =ti.MRI()
mri_dataset.normalize()
device = ti.torch.device('cuda:0')
model = ti.CNN().to(device)

AssertionError: Torch not compiled with CUDA enabled